In [ ]:
!pip install transformers

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import transformers
import torch
import re
import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.utils import shuffle

In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
df = pd.read_csv('/content/drive/My Drive/ML_models/cleaned_IMDB_Dataset.csv')

In [ ]:
df.head()

,review,sentiment
0,one review mention watch oz episod hook right ...,1
1,wonder littl product br br film techniqu unass...,1
2,thought wonder way spend time hot summer weeke...,1
3,basic famili littl boy jake think zombi closet...,0
4,petter mattei love time money visual stun film...,1


In [ ]:
df.columns = ['review','sentiment']

In [ ]:
X=list(df['review'])
y=list(df['sentiment'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, random_state = 0)

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(X_train, truncation=True, padding=True, return_tensors="tf")
test_encodings = tokenizer(X_test, truncation=True, padding=True, return_tensors="tf")

In [ ]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [ ]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'activation_13', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
model.fit(train_dataset.batch(16), epochs=3, batch_size=16)

Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

KeyboardInterrupt: ignored

In [ ]:
model.save_pretrained("/content/drive/My Drive/ML_models/saved_models/bert/bert_custom_model")

In [ ]:
loaded_model = TFDistilBertForSequenceClassification.from_pretrained("/content/drive/My Drive/ML_models/saved_models/bert/bert_custom_model")

Some layers from the model checkpoint at /content/drive/My Drive/ML_models/saved_models/bert/bert_custom_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/My Drive/ML_models/saved_models/bert/bert_custom_model and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it

In [ ]:
def get_predictions(test_sentence):
    predict_input = tokenizer.encode(test_sentence,
                                    truncation=True,
                                    padding=True,
                                    return_tensors="tf")
    tf_output = loaded_model.predict(predict_input)[0]
    tf_prediction = tf.nn.softmax(tf_output, axis=1).numpy()[0]
    if(tf_prediction[0]<tf_prediction[1]):
      return 1  # Positive
    else:
      return 0  # Negetive  

In [ ]:
prdicted_sentiment = list()
for sentence in X_test:
  prdicted_sentiment.append(get_predictions(sentence))

In [ ]:
print(classification_report(y_test, prdicted_sentiment, target_names=['negative_sentiment', 'positive_sentiment']))

                    precision    recall  f1-score   support

negative_sentiment       0.90      0.86      0.88      1014
positive_sentiment       0.87      0.90      0.88       986

          accuracy                           0.88      2000
         macro avg       0.88      0.88      0.88      2000
      weighted avg       0.88      0.88      0.88      2000



In [ ]:
# test_sentence = "I love Glasgow"
# get_predictions(test_sentence)